In [1]:
import os
from pathlib import Path
import ngauge
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict, deque, Counter
from brainlit.map_neurons.utils import replace_root, ZerothFirstOrderNeuron
from scipy.interpolate import splprep, splev
from similaritymeasures import frechet_dist
from brainlit.map_neurons.map_neurons import (
    Diffeomorphism_Transform,
)
import pickle
from tqdm import tqdm
import pandas as pd
from statannotations.Annotator import Annotator

In [2]:
swc_dir = (
    "/Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/mouselight-swcs/swcs-1"
)
swc_dir = Path(swc_dir)


swc_files = os.listdir(swc_dir)
swc_files = [swc_dir / f for f in swc_files if ".swc" in f]

In [3]:
def check_duplicates_center(neuron):
    """Check if neuron has multiple nodes with same coordinate location, and shifts the neuron so the bounding box around the neuron is at the origin.

    Args:
        neuron (nGauge.neuron): Neuron to be processed.

    Raises:
        ValueError: If there are multiple nodes at the same coordinate.

    Returns:
        ngauge.Neuron: Centered neuron.
    """
    assert len(neuron.branches) == 1

    stack = []
    stack += [neuron.branches[0]]
    coords = []

    while len(stack) > 0:
        child = stack.pop()
        stack += child.children
        coords.append([child.x, child.y, child.z])

    # look for duplicates
    dupes = []
    seen = set()
    for coord in coords:
        coord = tuple(coord)
        if coord in seen:
            dupes.append(coord)
        else:
            seen.add(coord)

    # center coordinates
    if len(dupes) > 0:
        raise ValueError(f"Duplicate nodes")
    else:
        coords = np.array(coords)
        mx = np.amax(coords, axis=0)
        mn = np.amin(coords, axis=0)
        center = np.mean(np.array([mx, mn]), axis=0)
        stack = [neuron.branches[0]]

        while len(stack) > 0:
            child = stack.pop()
            stack += child.children

            child.x -= center[0]
            child.y -= center[1]
            child.z -= center[2]

    return neuron

In [4]:
transform_fname = swc_dir.parents[0] / f"exp-morpho-diffeo-320.pickle"

with open(transform_fname, "rb") as handle:
    transform_data = pickle.load(handle)
    xv = transform_data["xv"]
    phii = transform_data["phii"]

ct = Diffeomorphism_Transform(xv, phii)

for swc_file in swc_files:
    neuron = ngauge.Neuron.from_swc(swc_file)
    neuron = replace_root(neuron)
    neuron = check_duplicates_center(neuron)

    ZerothFirstOrderNeuron(neuron, da=ct, sampling=2, print_time=True)

0.15700697898864746 for zeroth order, 14.768487930297852 for first order
0.06550288200378418 for zeroth order, 10.22117829322815 for first order
0.0271909236907959 for zeroth order, 3.2476770877838135 for first order
0.025649309158325195 for zeroth order, 2.4064688682556152 for first order
0.02886199951171875 for zeroth order, 2.2702860832214355 for first order
0.013966083526611328 for zeroth order, 2.67225980758667 for first order
0.03781294822692871 for zeroth order, 3.986912965774536 for first order
0.07328104972839355 for zeroth order, 10.608613729476929 for first order
0.20352602005004883 for zeroth order, 12.700295209884644 for first order
0.06024503707885742 for zeroth order, 16.533468008041382 for first order
0.6692087650299072 for zeroth order, 51.36285996437073 for first order
0.08608508110046387 for zeroth order, 31.225031852722168 for first order
0.2987799644470215 for zeroth order, 54.69789910316467 for first order
0.0520930290222168 for zeroth order, 19.959741830825806 fo

In [7]:
z = [
    0.15700697898864746,
    0.06550288200378418,
    0.0271909236907959,
    0.025649309158325195,
    0.02886199951171875,
    0.013966083526611328,
    0.03781294822692871,
    0.07328104972839355,
    0.20352602005004883,
    0.06024503707885742,
    0.6692087650299072,
    0.08608508110046387,
    0.2987799644470215,
    0.0520930290222168,
    0.09675478935241699,
    0.38361072540283203,
    0.030991315841674805,
    0.058589935302734375,
    0.06856918334960938,
    0.14633512496948242,
]
f = [
    14.768487930297852,
    10.22117829322815,
    3.2476770877838135,
    2.4064688682556152,
    2.2702860832214355,
    2.67225980758667,
    3.986912965774536,
    10.608613729476929,
    12.700295209884644,
    16.533468008041382,
    51.36285996437073,
    31.225031852722168,
    54.69789910316467,
    19.959741830825806,
    13.551981925964355,
    93.57587313652039,
    5.724336862564087,
    14.892866134643555,
    9.532119035720825,
    16.985495805740356,
]

assert len(z) == len(f)

print(np.mean(z))
print(np.mean(f))

0.12920305728912354
19.5461926817894
